In [1]:
import pytorch_lightning as pl
print(pl.__version__)

2023-01-27 11:08:46.497251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 11:08:47.537272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

1.8.6


In [2]:
import torch
print(torch.__version__)

1.13.1


In [3]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np


In [4]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from pytorch_lightning.callbacks import ModelCheckpoint

In [5]:
from transformers import (
     AdamW,
     T5ForConditionalGeneration,
     T5Tokenizer,
     get_linear_schedule_with_warmup
 )
# Seeds all the processes including numpy torch and other imported modules.
pl.seed_everything(0)

Global seed set to 0


0

In [6]:
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

### Data extraction and loading

In [7]:
data = pd.read_csv("documentation.csv")

In [8]:
data.columns

Index(['document', 'questions'], dtype='object')

In [12]:
data.document.nunique()

346

In [15]:
data.columns = ["pt_answer", "question"]

In [16]:
data.loc[0:10, "question"]

0                                  What does Dict() do?
1     What does 'OrderedDict' do to 'int, tVal' coll...
2                What does Dict[int, t] collections do?
3     What is the name of the function that enables ...
4                           How does the function work?
5     What is the name of the dict that -> Dict[str,...
6                                      Tensor builtins.
7                         -> Dict[str, tVal] built ins?
8     What is the name of the dict that is used to b...
9     What is the name of the dict that is built in ...
10                 What type of builtins does dict use?
Name: question, dtype: object

In [17]:
# import re
# CLEANR = re.compile('<.*?>') 

# def cleanhtml(raw_html):
#   cleantext = re.sub(CLEANR, '', raw_html)
#   return cleantext

In [18]:
# data["question"] = data["question"].apply(lambda x: cleanhtml(x))

In [19]:
# data.loc[0, "context"]

In [20]:
# data["context"] = data["context"].apply(lambda x: cleanhtml(x))

In [21]:
# data["pt_answer"] = data["pt_answer"].apply(lambda x: cleanhtml(x))

In [22]:
# data["pt_answer"] = data["pt_answer"] + data["context"]

### Tokenization

In [23]:
MODEL_NAME ='t5-base' 
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Create Dataset

In [42]:
class SODataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        source_max_token_len: int = 1024,
        target_max_token_len: int = 1024,
    ):
        self.data = data
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        source_encoding = tokenizer(
            data_row["question"],
#             data_row["context"],
            max_length=self.source_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )
        target_encoding = tokenizer(
            data_row["pt_answer"],
            max_length=self.target_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )
        labels = target_encoding["input_ids"]
        labels[labels == 0] = -100
        return dict(
            question=data_row["question"],
#             context=data_row["context"],
            answer_text=data_row["pt_answer"],
            input_ids=source_encoding["input_ids"].flatten(),
            attention_mask=source_encoding["attention_mask"].flatten(),
            labels=labels.flatten(),
        )

In [43]:
sample_dataset = SODataset(data, tokenizer)


### Split data into train and validation

In [44]:
# data = data[:1000]

In [45]:
train_df, val_df = train_test_split(data, test_size=0.05)
train_df.shape,  val_df.shape 

((5043, 2), (266, 2))

### Create PTL Data module class

In [46]:
class SODataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 4,
        source_max_token_len: int = 1024,
        target_max_token_len: int = 1024,
    ):
        super().__init__()
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self, stage):
        self.train_dataset = SODataset(
            self.train_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len
        )
        self.test_dataset = SODataset(
            self.test_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4
        )

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=1, num_workers=4)


In [47]:
BATCH_SIZE = 1
N_EPOCHS = 1
data_module = SODataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup("fit")

### Loading T5 pretrained model

In [48]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [49]:
# model = AutoModelForSeq2SeqLM.from_pretrained("t5-base", return_dict=True)

In [50]:
# ?model

### Model finetuning

In [51]:
class SOModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-base", return_dict=True)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
#         labels = input_ids
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
#         labels = input_ids
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
#         labels = input_ids
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=0.0001)
        return optimizer

In [52]:
model = SOModel()

In [53]:
# model

In [54]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [55]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=3)
# logger = TensorBoardLogger("training-logs", name="bio-qa")
logger = TensorBoardLogger("training-logs", name="so-qa")
trainer = pl.Trainer(
    logger = logger,
#     callbacks = [checkpoint_callback],
    callbacks = [checkpoint_callback, early_stopping],
    max_epochs=50,
    gpus=8)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [56]:
# ! pip install -q tensorboard

In [57]:
# %load_ext tensorboard
# %tensorboard --logdir ./training-logs

In [58]:
import transformers
transformers.logging.set_verbosity_error()

In [59]:
trainer.fit(model, data_module)

Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/8
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/8
Initializing distributed: GLOBAL_RANK: 2, MEMBER: 3/8
Initializing distributed: GLOBAL_RANK: 3, MEMBER: 4/8
Initializing distributed: GLOBAL_RANK: 4, MEMBER: 5/8
Initializing distributed: GLOBAL_RANK: 5, MEMBER: 6/8
Initializing distributed: GLOBAL_RANK: 6, MEMBER: 7/8
Initializing distributed: GLOBAL_RANK: 7, MEMBER: 8/8
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 8 processes
----------------------------------------------------------------------------------------------------

/opt/conda/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /home/ubuntu/shrinath/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
L

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:208: UserWarning: num_workers>0, persistent_workers=False, and strategy=ddp_spawn may result in data loading bottlenecks. Consider setting persistent_workers=True (this is a limitation of Python .spawn() and PyTorch)
  rank_zero_warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:537: PossibleUserWarning: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

[W reducer.cpp:1298] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
[W reducer.cpp:1298] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later 

...

Epoch 47, global step 30288: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 48, global step 30919: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 49, global step 31550: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=50` reached.


In [60]:
# trainer.save_checkpoint("checkpoints/epoch50.ckpt")

In [61]:
# trainer.test(model, data_module)

### test against validation dataset

In [81]:
def generate_answer(question, best_model):
    source_encoding = tokenizer(
        question["pt_title"],
#         question["pt_body"],
        max_length=1024,
        padding="max_length",
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    )
    generated_ids = best_model.model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"],
        num_beams=1,  # greedy search
        max_length=512,
        repetition_penalty=2.5,
        early_stopping=True,
        use_cache=True,
    )
#     print(generated_ids)
    preds = [
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    ]
    return "".join(preds)

In [63]:
# model = SOModel()
# # best_model = model.load_from_checkpoint("checkpoints/best-checkpoint-epoch=18-val_loss=1.55.ckpt")
# best_model = model.load_from_checkpoint("checkpoints/epoch50.ckpt")

In [84]:
best_model = trainer.model

In [85]:
question = {}

question["pt_title"] = "Saving and loading models"

generate_answer(question, best_model)

'This module contains observers which are used to collect statistics about the values observed during calibration (PTQ) or training (QAT). +------------+--------------------------------------------------------------------------------------------+ | "Observer | Base observer Module. | | Base" | | +------------+--------------------------------------------------------------------------------------------+ | "MinMaxOb | Observer module for computing the quantization parameters based on the running min and max | | server" | values. | +------------+--------------------------------------------------------------------------------------------+ | "MovingAv | Observer module for computing the quantization parameters based on the moving average of | | erageMinM | the min and max values. | | axObserve | | | r" | | +------------+--------------------------------------------------------------------------------------------+ | "PerChann | Observer module for computing the quantization parameters based on

In [86]:
question = {}

question["pt_title"] = "what is torch.storage?"

generate_answer(question, best_model)

'See what is inside a package? ----------------------------- Treat the packages like ZIP archives  The container format for a "torch.package" is zip, so any tools that work with standard ZipFile are available to view and use. Some common ways to interact with ZIP files: * "unzip my_package.pt" will unzip the "torch.package" archive to disk, where you can freely inspect its contents. $ unzip my_package.pt && tree my_package my_package .data   94304870911616.storage   92304900784016.storage   extern_modules   version  models   model_1.pkl  torchvision  models  resnet.py  utils.py  cd my_package && cat torchvision/models/resnet.py... * The Python "zipfile" module provides a standard way to read and write ZIP archive contents. from zipfile import ZipFile with ZipFile("my_package.pt") as myzip: file_bytes = myzip.read("torchvision/models/resnet.py") # edit file_bytes in some way myzip.writestr("torchvision/models/resnet.py", new_file_bytes) * vim has the ability to natively read ZIP archive

In [87]:
question = {}

question["pt_title"] = "How to load model in PyTorch?"


In [88]:
generate_answer(question, best_model)

'+------------+--------------------------------------------------------------------------------------------+ | "script" | Scripting a function or "nn.Module" will inspect the source code, compile it as | | | TorchScript code using the TorchScript compiler, and return a "ScriptModule" or | | | "ScriptFunction". | +------------+--------------------------------------------------------------------------------------------+ | "trace" | Trace a function and Returns an executable or "ScriptFunction" that will be optimized | | | using just-in-time compilation. | +------------+--------------------------------------------------------------------------------------------+ | "script_i | Compilerates "fn" when it is first called during tracing. | | f_tracing" | | +------------+--------------------------------------------------------------------------------------------+ | "trace_mo | Trace a module and return an executable "ScriptModule" that will be optimised using just- | | dule" | in-time compilati

In [89]:
question = {}

question["pt_title"] = "How to crop an image using pytorch?"

question["pt_body"] = """
How to crop an image using pytorch?

"""

generate_answer(question, best_model)

'+------------+--------------------------------------------------------------------------------------------+ | "pixel_sh | Rearranges elements in a tensor of shape (*, C times r2, H, W) to a tensor of shape (*, | | uffle" | C, H times r, W times r), where r is the "upscale_factor". | +------------+--------------------------------------------------------------------------------------------+ | "pixel_un | Reverses the "PixelShuffle" operation by rearranging elements in a tensor of shape (*, C, | | shuffle" | H times r, W times r) to a tensor of shape (*, C times r3, H, W), where r is the | | | "downscale_factor". | +------------+--------------------------------------------------------------------------------------------+ | "pad" | Pads tensor. | +------------+--------------------------------------------------------------------------------------------+ | "interpol | Down/up samples the input to either the given "size" or the given "scale_factor" | | ate" | | +------------+----------------

In [90]:
question = {}

question["pt_title"] = "How to convert float tensor to int?"

question["pt_body"] = """
How to convert float tensor to int?

"""

generate_answer(question, best_model)

"TorchScript supports a subset of Python's variable resolution (i.e. scoping) rules. Local variables behave the same as in Python, except for the restriction that a variable must have the same type along all paths through a function. If a variable has a different type on different branches of an if statement, it is an error to use it after the end of the if statements. Similarly, a variable is not allowed to be used if it is only *defined* along some paths through the function. Example: @torch.jit.script def foo(x): if x  0: y = 4 print(y) Traceback (most recent call last):... RuntimeError:... y is not defined in the false branch... @torch.jit.script... def foo(x): if x  0:  y = 4  --- HERE print(y) and was used here: if x  0: y = 4 Print(y)  --- HERE... Non-local variables are resolved to Python values at compile time when the function is defined. These values are then converted into TorchScript values using the rules described in Use of Python Values. Use of Python Values"